https://portal.nccs.nasa.gov/datashare/gmao/geos-cf/v1/das/Y2024/M10/D04/
NASA Center for Climate Simulation
https://www.nccs.nasa.gov/services/climate-data-services

### METEO


historico = https://open-meteo.com/en/docs/historical-weather-api
forecasted = https://open-meteo.com/en/docs

pip install openmeteo-requests

pip install requests-cache retry-requests numpy pandas

###### san juan 
	"latitude": -17.507429,
	"longitude": -62.843276,
###### pozo tigre
 -17.627575, -61.979184
###### Tres cruces
 -17.638442, -62.260282
###### Guarayo
 -17.574156, -61.831043


 
//////// NO: # san miguel mas al sur
-17.310852, -61.426725



In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

ModuleNotFoundError: No module named 'openmeteo_requests'

In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


In [ ]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
 
params = {
	"latitude": -17.638442,
	"longitude": -62.260282,
	"start_date": "2020-01-01",
	"end_date": "2024-10-05",
	"daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "precipitation_sum", "rain_sum", "precipitation_hours", "shortwave_radiation_sum", "et0_fao_evapotranspiration"],
	"timezone": "America/New_York"
}
responses = openmeteo.weather_api(url, params=params)


In [ ]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")


Coordinates -17.60984230041504°N -62.234039306640625°E
Elevation 281.0 m asl
Timezone b'America/New_York' b'EDT'
Timezone difference to GMT+0 -14400 s


In [ ]:
daily

NameError: name 'daily' is not defined

### Info historica

In [ ]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(3).ValuesAsNumpy()
daily_rain_sum = daily.Variables(4).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(5).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(6).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(7).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

daily_dataframe = pd.DataFrame(data = daily_data)
# print(daily_dataframe)

In [ ]:
daily_dataframe

,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,rain_sum,precipitation_hours,shortwave_radiation_sum,et0_fao_evapotranspiration
0,2020-01-01 04:00:00+00:00,31.669500,22.869501,26.632002,0.6,0.6,3.0,20.340000,4.346330
1,2020-01-02 04:00:00+00:00,30.369501,23.169500,26.698664,5.5,5.5,5.0,17.629999,3.728637
2,2020-01-03 04:00:00+00:00,32.719501,24.019501,28.157000,0.5,0.5,3.0,25.639999,5.601533
3,2020-01-04 04:00:00+00:00,33.269501,24.019501,28.646584,0.0,0.0,0.0,27.549999,6.145042
4,2020-01-05 04:00:00+00:00,32.919502,24.369501,28.444504,0.3,0.3,1.0,25.309999,6.205818
...,...,...,...,...,...,...,...,...,...
1735,2024-10-01 04:00:00+00:00,38.019501,27.769501,32.519505,0.0,0.0,0.0,21.629999,9.192921
1736,2024-10-02 04:00:00+00:00,36.869499,27.469501,31.996588,0.0,0.0,0.0,19.799999,7.415186
1737,2024-10-03 04:00:00+00:00,33.719501,24.219501,NaN,NaN,NaN,0.0,NaN,NaN
1738,2024-10-04 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


####  añadir un lag a la historica para tener temperatura predicha y precipitacion de 16 dias

In [ ]:
# Convert the date column to datetime format
daily_dataframe['date'] = pd.to_datetime(daily_dataframe['date'])
print(len(daily_dataframe))


1740


In [ ]:


# Create lag columns for the next 16 days for temperature max, temperature min, and precipitation sum
daily_dataframe['delta_hoy'] = daily_dataframe['temperature_2m_max']-daily_dataframe['temperature_2m_min']
daily_dataframe['temp_max_16days'] = daily_dataframe['temperature_2m_max'].shift(-16)
daily_dataframe['temp_min_16days'] = daily_dataframe['temperature_2m_min'].shift(-16)
daily_dataframe['delta_16dias'] = daily_dataframe['temp_max_16days']-daily_dataframe['temp_min_16days']

daily_dataframe['precip_16days'] = daily_dataframe['precipitation_sum'].shift(-16)
daily_dataframe['precipitation_hours_16days'] = daily_dataframe['precipitation_sum'].shift(-16)


# Display the result
daily_dataframe.head(30)

,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,rain_sum,precipitation_hours,shortwave_radiation_sum,et0_fao_evapotranspiration,delta_hoy,temp_max_16days,temp_min_16days,delta_16dias,precip_16days,precipitation_hours_16days
0,2020-01-01 04:00:00+00:00,31.669500,22.869501,26.632002,0.600000,0.600000,3.0,20.340000,4.346330,8.799999,32.969501,24.069500,8.900002,4.300000,4.300000
1,2020-01-02 04:00:00+00:00,30.369501,23.169500,26.698664,5.500000,5.500000,5.0,17.629999,3.728637,7.200001,33.469501,24.669500,8.800001,2.200000,2.200000
2,2020-01-03 04:00:00+00:00,32.719501,24.019501,28.157000,0.500000,0.500000,3.0,25.639999,5.601533,8.700001,35.569500,25.219501,10.349998,0.100000,0.100000
3,2020-01-04 04:00:00+00:00,33.269501,24.019501,28.646584,0.000000,0.000000,0.0,27.549999,6.145042,9.250000,35.069500,24.869501,10.199999,0.300000,0.300000
4,2020-01-05 04:00:00+00:00,32.919502,24.369501,28.444504,0.300000,0.300000,1.0,25.309999,6.205818,8.550001,34.419502,25.569500,8.850002,4.100000,4.100000
5,2020-01-06 04:00:00+00:00,31.219501,25.019501,27.717421,1.700000,1.700000,9.0,21.410000,5.700644,6.200001,32.519501,22.819500,9.700001,44.699997,44.699997
6,2020-01-07 04:00:00+00:00,26.919500,23.219501,24.415335,41.399998,41.399998,21.0,7.890000,1.735862,3.699999,35.019501,23.569500,11.450001,0.200000,0.200000
7,2020-01-08 04:00:00+00:00,28.419500,22.319500,25.388250,8.400001,8.400001,11.0,16.400000,3.662186,6.100000,34.419502,23.469501,10.950001,0.000000,0.000000
8,2020-01-09 04:00:00+00:00,29.519501,24.719501,27.094503,4.400000,4.400000,11.0,21.780001,5.204575,4.799999,35.769501,21.369501,14.400000,0.000000,0.000000
9,2020-01-10 04:00:00+00:00,31.719501,24.969501,28.082003,0.600000,0.600000,4.0,24.910000,6.217043,6.750000,35.269501,24.619501,10.650000,0.300000,0.300000


In [ ]:
# Remove timezone information from datetime columns
daily_dataframe['date'] = daily_dataframe['date'].dt.tz_localize(None)

# Save to Excel
daily_dataframe.to_excel('DATOSHISTORICOStresCruces.xlsx', index=False)
# daily_dataframe.to_excel('DATOSHISTORICOS.xlsx', index=False)

### Forecast

In [ ]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(2).ValuesAsNumpy()
daily_rain_sum = daily.Variables(3).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(4).ValuesAsNumpy()
daily_precipitation_probability_max = daily.Variables(5).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(6).ValuesAsNumpy()
daily_et0_fao_evapotranspiration = daily.Variables(7).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["precipitation_probability_max"] = daily_precipitation_probability_max
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

daily_dataframe_forecast = pd.DataFrame(data = daily_data)
print(daily_dataframe_forecast)

                        date  temperature_2m_max  temperature_2m_min  \
0  2024-09-19 04:00:00+00:00           23.708500           14.058500   
1  2024-09-20 04:00:00+00:00           22.858500           11.858500   
2  2024-09-21 04:00:00+00:00           22.908501           10.958500   
3  2024-09-22 04:00:00+00:00           23.958500           10.258500   
4  2024-09-23 04:00:00+00:00           24.258501            9.458500   
5  2024-09-24 04:00:00+00:00           17.658501           11.908501   
6  2024-09-25 04:00:00+00:00           19.158501           13.058500   
7  2024-09-26 04:00:00+00:00           19.458500           14.708500   
8  2024-09-27 04:00:00+00:00           19.408501           12.808500   
9  2024-09-28 04:00:00+00:00           15.258500            6.408500   
10 2024-09-29 04:00:00+00:00           14.258500            5.908500   
11 2024-09-30 04:00:00+00:00           15.108500            6.408500   
12 2024-10-01 04:00:00+00:00           13.258500            8.40